In [10]:
!pip3 install ray

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
import ray
import random, logging
import xml.etree.ElementTree as etree


In [12]:
def parse_post(xml):
    return etree.fromstring(xml)

In [13]:
posts = [
  '<row Id="0" Title="Eliciting priors from experts" />',
  '<row Id="2" Title="What is normality?" />',
  '<row Id="3" Title="What are some valuable Statistical Analysis open source projects?" />',
  '<row Id="4" Title="Assessing the significance of differences in distributions" />',
  '<row Id="5" Title="The Two Cultures: statistics vs. machine learning?" />',
  '<row Id="6" Title="Locating freely available data samples" />',
  '<row Id="7" Title="Forecasting demographic census" />',
  '<row Id="8" Title="Multivariate Interpolation Approaches" />',
  '<row Id="9" Title="How can I adapt ANOVA for binary data?" />'
]

In [14]:
[ parse_post(xml) for xml in posts ]

[<Element 'row' at 0x1178166d0>,
 <Element 'row' at 0x117816a40>,
 <Element 'row' at 0x117816a90>,
 <Element 'row' at 0x117816ae0>,
 <Element 'row' at 0x117816b30>,
 <Element 'row' at 0x117816b80>,
 <Element 'row' at 0x117816c20>,
 <Element 'row' at 0x117816c70>,
 <Element 'row' at 0x117816cc0>]

In [15]:
def parse_post(xml):
    post = etree.fromstring(xml)
    print(post.get('Id'))
    return post

In [6]:
[ parse_post(xml) for xml in posts ]

1
2
3
4
5
6
7
8
9


[<Element 'row' at 0x7efdd8eeb450>,
 <Element 'row' at 0x7efdd8eeb590>,
 <Element 'row' at 0x7efdd8eeba90>,
 <Element 'row' at 0x7efdf16f29f0>,
 <Element 'row' at 0x7efdd8eebae0>,
 <Element 'row' at 0x7efdd8eebb30>,
 <Element 'row' at 0x7efdd8eebbd0>,
 <Element 'row' at 0x7efdd8eebc20>,
 <Element 'row' at 0x7efdd8eebc70>]

In [16]:
# Start Ray. If you're connecting to an existing cluster, you would use
# ray.init(address=<cluster-address>) instead.
ray.init(logging_level=logging.ERROR)

Python version:,3.9.6
Ray version:,2.0.0


In [17]:
@ray.remote
def parse_post(xml):
    post = etree.fromstring(xml)
    print(post.get('Id'))
    return post

In [18]:
future = parse_post.remote(posts[0])

(parse_post pid=27486) 0


In [19]:
ray.get(future)

<Element 'row' at 0x11784f770>

In [21]:
futures = [parse_post.remote(xml) for xml in posts ]

(parse_post pid=27486) 0
(parse_post pid=27486) 4
(parse_post pid=27483) 5
(parse_post pid=27482) 6
(parse_post pid=27479) 7
(parse_post pid=27480) 3
(parse_post pid=27477) 8
(parse_post pid=27485) 2
(parse_post pid=27481) 9


In [22]:
futures

[ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000),
 ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000),
 ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000),
 ObjectRef(e0dc174c83599034ffffffffffffffffffffffff0100000001000000),
 ObjectRef(f4402ec78d3a2607ffffffffffffffffffffffff0100000001000000),
 ObjectRef(f91b78d7db9a6593ffffffffffffffffffffffff0100000001000000),
 ObjectRef(82891771158d68c1ffffffffffffffffffffffff0100000001000000),
 ObjectRef(8849b62d89cb30f9ffffffffffffffffffffffff0100000001000000),
 ObjectRef(80e22aed7718a125ffffffffffffffffffffffff0100000001000000)]

In [23]:
ray.get(futures)

[<Element 'row' at 0x1660139f0>,
 <Element 'row' at 0x166021220>,
 <Element 'row' at 0x166021310>,
 <Element 'row' at 0x1660212c0>,
 <Element 'row' at 0x166021360>,
 <Element 'row' at 0x166021400>,
 <Element 'row' at 0x166021450>,
 <Element 'row' at 0x166021770>,
 <Element 'row' at 0x1660216d0>]

In [24]:
[ el.get('Id') for el in ray.get(futures) ]

['0', '2', '3', '4', '5', '6', '7', '8', '9']

In [25]:
# similar to rdd.cache()
ref = ray.put("JP")

In [26]:
ray.get(ref)

'JP'

In [27]:
ref

ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000)

## Actors

Scheme made them [concrete](https://dspace.mit.edu/handle/1721.1/5794). Erlang made them [useful](https://erlang.org/doc/getting_started/conc_prog.html). Akka made them [cool](https://akka.io/). And now Ray makes them [easy](https://docs.ray.io/en/latest/ray-overview/index.html)!

In [26]:
!pip install faker

In [27]:
@ray.remote
class Child(object):
    def __init__(self):
        from faker import Faker
        self.name = Faker().name()
        self.age = 1
        
    def grow(self):
        self.age += 1
        return self.age
    
    def greet(self):
        return (
            f'My name is {self.name} '
            f'and I am {self.age} years old'
        )

In [28]:
children = [Child.remote() for i in range(10)]

In [29]:
children

[Actor(Child,3db7cfef01000000),
 Actor(Child,a628090a01000000),
 Actor(Child,fafba2ba01000000),
 Actor(Child,b7603b6c01000000),
 Actor(Child,a491754501000000),
 Actor(Child,84b65a9401000000),
 Actor(Child,87b4f72601000000),
 Actor(Child,be3cb80901000000),
 Actor(Child,bd5c534001000000),
 Actor(Child,7f10737001000000)]

In [30]:
futures = [ c.greet.remote() for c in children ]

In [31]:
futures

[ObjectRef(b19fee1fe487d3333db7cfef0100000001000000),
 ObjectRef(af23404c2bbc23f5a628090a0100000001000000),
 ObjectRef(99506fd459680ea2fafba2ba0100000001000000),
 ObjectRef(c61c90f84b215448b7603b6c0100000001000000),
 ObjectRef(11d7f983e14011f4a49175450100000001000000),
 ObjectRef(7d269126652173d084b65a940100000001000000),
 ObjectRef(a16678a371591cc487b4f7260100000001000000),
 ObjectRef(82be70daa7562412be3cb8090100000001000000),
 ObjectRef(d24da3c30343f5b0bd5c53400100000001000000),
 ObjectRef(335d6f0817bbad8f7f1073700100000001000000)]

In [32]:
for future in ray.get(futures):
    print(future)

My name is Brenda Wilson and I am 1 years old
My name is Barbara Morgan and I am 1 years old
My name is Christina Santos and I am 1 years old
My name is Eddie Moran and I am 1 years old
My name is Kevin Robinson and I am 1 years old
My name is Tony Myers and I am 1 years old
My name is Diane Clark and I am 1 years old
My name is Cindy Dawson and I am 1 years old
My name is George Leonard and I am 1 years old
My name is Teresa Mendoza and I am 1 years old


In [33]:
for c in children:
    for _ in range(random.randint(1,10)):
        c.grow.remote()

In [34]:
for future in ray.get([ c.greet.remote() for c in children ]):
    print(future)

My name is Brenda Wilson and I am 9 years old
My name is Barbara Morgan and I am 10 years old
My name is Christina Santos and I am 6 years old
My name is Eddie Moran and I am 9 years old
My name is Kevin Robinson and I am 8 years old
My name is Tony Myers and I am 7 years old
My name is Diane Clark and I am 5 years old
My name is Cindy Dawson and I am 2 years old
My name is George Leonard and I am 6 years old
My name is Teresa Mendoza and I am 11 years old


In [35]:
c = children[0]

In [38]:
ray.get([c.grow.remote() for _ in range(5)])

[10, 11, 12, 13, 14]

In [39]:
# actors stay around as long as they are in scope
# since nothing really goes out of scope in a notebook
# we have to manually terminate them
[ ray.kill(person) for person in children ]

[None, None, None, None, None, None, None, None, None, None]

In [40]:
ray.shutdown()

## Simulating a pandemic

> note this is a toy model simulation, results should not be used to inform health decisions or personal behavior

### The SIR epidemic model:

$S(t)$: susceptible individuals who have not yet been infected at time $t$

$I(t)$: number of infectious individuals at time $t$

$R(t)$: number of individuals who have recovered (and are immune) at time $t$

#### Parameters

$\beta$: probablity of transmitted the disease from an infected to a susceptible individual

$\gamma$: recovery rate ~ $\frac{1}{\text{duration of disease}}$

We will follow the [EMOD compartamental model](https://idmod.org/docs/emod/malaria/model-compartments.html) to simulate the SIR model as a series of discrete timesteps. For something like reinforcement learning, instead of disease dynamics you simulate actions in an environment/game.

In [41]:
ray.init(logging_level=logging.ERROR)

{'node_ip_address': '172.17.0.2',
 'raylet_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:6379',
 'object_store_address': '/tmp/ray/session_2021-02-19_02-30-02_212321_7103/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-19_02-30-02_212321_7103/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-19_02-30-02_212321_7103',
 'metrics_export_port': 53482,
 'node_id': '21ec551dd6b0445306d5d05b9ac62ba9c2239269'}

In [42]:
# parameters
b = 0.5
b_0 = 0.2
g = 0.2
dim = 5

In [50]:
@ray.remote
class Person(object):
    def __init__(self, i):
        self.index = i
        self.state = 'i' if random.random() < b_0 else 's'
        self.x = random.randint(0, dim)
        self.y = random.randint(0, dim)
        
    def location(self):
        return (self.x, self.y)
    
    def health(self):
        return self.state
    
    def index(self):
        return self.index
    
    def status(self):
        return f"Individual {self.index} at {self.location()} is currently {self.state}"
       
    def walk(self):
        if self.state == 'i':
            if random.random() < g:
                print(f"{self.index} has recovered ⚕️")
                self.state = 'r'

        self.x += random.randint(-1, 1)
        self.y += random.randint(-1, 1)
        
        self.x = max(min(self.x, dim), 0)
        self.y = max(min(self.y, dim), 0)
        
    def contract(self):
        print(f"{self.index} has become sick 🤮")
        self.state = 'i'
        
    def interact(self, stranger):
        x, y = ray.get(stranger.location.remote())
        state = ray.get(stranger.health.remote())
        
        # is the stranger close to me
        if (abs(x - self.x) <= 1) and (abs(y - self.y) <= 1):
            # is either of us infected?
            if self.state == 'i' or state == 'i':
                # can either of us _get_ infected?
                if self.state == 's' or state == 's':
                    # which one of us can get the disease
                    contract = self.contract if self.state == 's' else stranger.contract.remote
                    
                    # roll the dice babeeeeee
                    if random.random() < b:
                        contract()

In [51]:
people = [Person.remote(i) for i in range(15)]

In [52]:
people

[Actor(Person,24ca2d6d01000000),
 Actor(Person,f26277e401000000),
 Actor(Person,3abfa87701000000),
 Actor(Person,e73a610901000000),
 Actor(Person,18b0341001000000),
 Actor(Person,c8c3554001000000),
 Actor(Person,cdeac00601000000),
 Actor(Person,25eb8b3501000000),
 Actor(Person,9cedd66601000000),
 Actor(Person,362823fd01000000),
 Actor(Person,b04ed06d01000000),
 Actor(Person,d6a2003a01000000),
 Actor(Person,8d83385001000000),
 Actor(Person,ccbb223201000000),
 Actor(Person,e804d33501000000)]

In [53]:
ray.get([p.location.remote() for p in people])

[(4, 2),
 (5, 1),
 (0, 4),
 (5, 2),
 (5, 5),
 (0, 0),
 (4, 1),
 (3, 4),
 (1, 4),
 (1, 5),
 (3, 5),
 (5, 2),
 (1, 1),
 (5, 2),
 (5, 3)]

In [54]:
ray.get([p.health.remote() for p in people])

['s', 'i', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 'i', 's']

In [55]:
from itertools import combinations

In [56]:
for i in range(20):
    print(f'\nIteration {i}\n\n')
    for person in people:
        person.walk.remote()
        
    pairs = list(combinations(people, 2))
    
    for p1, p2 in pairs:
        p1.interact.remote(p2)


Iteration 0



Iteration 1


(pid=7932) 3 has become sick 🤮
(pid=7984) 6 has become sick 🤮
(pid=7908) 0 has become sick 🤮
(pid=7956) 4 has become sick 🤮
(pid=8059) 11 has become sick 🤮
(pid=8099) 14 has become sick 🤮

Iteration 2


(pid=8059) 11 has recovered ⚕️
(pid=8099) 14 has recovered ⚕️
(pid=7909) 1 has recovered ⚕️
(pid=7956) 4 has recovered ⚕️

Iteration 3


(pid=7984) 6 has recovered ⚕️

Iteration 4


(pid=7967) 5 has become sick 🤮

Iteration 5



Iteration 6



Iteration 7


(pid=7932) 3 has recovered ⚕️
(pid=8014) 8 has become sick 🤮
(pid=8014) 8 has recovered ⚕️
(pid=8069) 12 has become sick 🤮

Iteration 8



Iteration 9


(pid=8069) 12 has recovered ⚕️
(pid=7967) 5 has recovered ⚕️

Iteration 10


(pid=8089) 13 has recovered ⚕️

Iteration 11



Iteration 12



Iteration 13



Iteration 14



Iteration 15



Iteration 16



Iteration 17



Iteration 18



Iteration 19


(pid=8044) 10 has become sick 🤮
(pid=7908) 0 has recovered ⚕️


In [57]:
for person in people:
    print(ray.get(person.status.remote()))

Individual 0 at (4, 3) is currently r
Individual 1 at (0, 3) is currently r
Individual 2 at (3, 5) is currently s
Individual 3 at (5, 1) is currently r
Individual 4 at (0, 5) is currently r
Individual 5 at (1, 5) is currently r
Individual 6 at (5, 0) is currently r
Individual 7 at (0, 4) is currently s
Individual 8 at (1, 2) is currently r
Individual 9 at (2, 5) is currently s
Individual 10 at (3, 0) is currently i
Individual 11 at (4, 3) is currently r
Individual 12 at (5, 5) is currently r
Individual 13 at (0, 1) is currently r
Individual 14 at (2, 2) is currently r
